<a href="https://colab.research.google.com/github/zhang8yiming/ML-DL/blob/main/L8_Homework_Reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN 文本分类

## 作业内容

本次作业，你将：

- 阅读文本分类的代码，找到可以替换 RNN 层的位置，用 LSTM 层进行替换；

- 将 LSTM 设置为双向 LSTM，修改后续对应层的维度，从新训练模型，记录验证集精度。

## 数据简介

- 数据集包含用户与音箱等智能设备进行单轮对话的文本数据，包含共 12 个意图类别。

- 数据以 CSV 格式存储，包括 train.csv、test.csv 和 test_label.csv 三个文件。

- train.csv 为训练数据，共约 12100 条，其文本由ASR语音识别技术转写得到，因此可能需要进一步处理，文本标注包含用户意图。

- test.csv 、test_label.csv 为测试数据，共约 3000 条。

In [ ]:
import torch
import numpy as np
import jieba
import pandas as pd

train_data = pd.read_csv('对话意图分类/train.csv', sep='\t', header=None)
test_data = pd.read_csv('对话意图分类/test.csv', sep='\t', header=None)

In [ ]:
train_data.head()

,0,1
0,还有双鸭山到淮阴的汽车票吗13号的,Travel-Query
1,从这里怎么回家,Travel-Query
2,随便播放一首专辑阁楼里的佛里的歌,Music-Play
3,给看一下墓王之王嘛,FilmTele-Play
4,我想看挑战两把s686打突变团竞的游戏视频,Video-Play


In [ ]:
train_data = train_data.sample(frac=1.0)

In [ ]:
train_data[1], lbl = pd.factorize(train_data[1])

In [ ]:
def coustom_data_iter(texts, labels):
    for x, y in zip(texts, labels):
        yield x, y

In [ ]:
train_iter = coustom_data_iter(train_data[0].values[:], train_data[1].values[:])

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = jieba.lcut


def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
vocab.get_itos()[:10]

['<unk>', '的', '我', '一下', '播放', '是', '吗', '给', '帮', '一个']

In [ ]:
vocab(['我', '一下', '今天'])

[2, 3, 41]

In [ ]:
def text_pipeline(x): return vocab(tokenizer(x))

In [ ]:
processed_text = torch.tensor(text_pipeline('今天我们在这里'), dtype=torch.int64)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

device = 'cpu'


def collate_batch(batch, max_len=20):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        processed_text = F.pad(processed_text, pad=[0, max_len,], mode='constant', value=0)
        if len(processed_text) > max_len:
            processed_text = processed_text[:max_len]

        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list).T
    return label_list.to(device), text_list.to(device)

In [ ]:
class BILSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size):
        super(BILSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        # self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_w2v))
        
        self.lstm = torch.nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=False)
        self.hidden2label = torch.nn.Linear(hidden_dim*1, label_size)
        
    def forward(self, sentence):
        # print(sentence.shape)
        sentence = torch.transpose(sentence, 1, 0)
        # print(sentence.shape)
        x = self.embeddings(sentence)
        # print(x.shape)
        lstm_out, self.hidden = self.lstm(x)
        # print(lstm_out.shape)
        y = self.hidden2label(lstm_out[-1,:,:])
        return y

In [ ]:
torch.transpose(torch.rand((2,3,1)), 0, 2).shape

torch.Size([1, 3, 2])

In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
num_class = len(lbl)
vocab_size = len(vocab)
emsize = 100
model = BILSTM(vocab_size, emsize, 64, num_class).to(device)

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10  # epoch
LR = 2 # learning rate
BATCH_SIZE = 16  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5.0, gamma=0.75)
total_accu = None

train_iter = coustom_data_iter(train_data[0].values[:], train_data[1].values[:])
train_dataset = to_map_style_dataset(train_iter)

num_train = int(len(train_dataset) * 0.75)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

In [ ]:
for (label, text) in train_dataloader:
    break
    
model(text).shape
# 12, 16, 100
# sequence length * batch size * embedding dim

torch.Size([16, 12])

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))

| end of epoch   1 | time:  3.17s | valid accuracy    0.100 
| end of epoch   2 | time:  3.12s | valid accuracy    0.104 
| end of epoch   3 | time:  3.08s | valid accuracy    0.106 
| end of epoch   4 | time:  3.08s | valid accuracy    0.117 
| end of epoch   5 | time:  3.07s | valid accuracy    0.116 
| end of epoch   6 | time:  3.09s | valid accuracy    0.124 
| end of epoch   7 | time:  3.08s | valid accuracy    0.106 
| end of epoch   8 | time:  3.10s | valid accuracy    0.135 
| end of epoch   9 | time:  3.10s | valid accuracy    0.129 
| end of epoch  10 | time:  3.10s | valid accuracy    0.181 


## 问题1：LSTM尝试

阅读上述文本分类的代码，找到可以替换LSTM层的位置进行替换。
- 替换之后模型的精度有提高吗？
- 替换之后模型的训练速度有提高吗？

In [ ]:
class BILSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size):
        super(BILSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        # self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_w2v))
        
        # 修改这一行
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=False)
        self.hidden2label = torch.nn.Linear(hidden_dim*1, label_size)
        
    def forward(self, sentence):
        # print(sentence.shape)
        sentence = torch.transpose(sentence, 1, 0)
        # print(sentence.shape)
        x = self.embeddings(sentence)
        # print(x.shape)
        lstm_out, self.hidden = self.lstm(x)
        # print(lstm_out.shape)
        y = self.hidden2label(lstm_out[-1,:,:])
        return y

In [ ]:
num_class = len(lbl)
vocab_size = len(vocab)
emsize = 100
model = BILSTM(vocab_size, emsize, 64, num_class).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5.0, gamma=0.75)

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))

| end of epoch   1 | time:  9.73s | valid accuracy    0.363 
| end of epoch   2 | time:  9.75s | valid accuracy    0.625 
| end of epoch   3 | time:  9.94s | valid accuracy    0.699 
| end of epoch   4 | time:  9.81s | valid accuracy    0.742 
| end of epoch   5 | time:  9.70s | valid accuracy    0.765 
| end of epoch   6 | time:  9.66s | valid accuracy    0.776 
| end of epoch   7 | time:  9.82s | valid accuracy    0.782 
| end of epoch   8 | time:  9.67s | valid accuracy    0.797 
| end of epoch   9 | time:  9.73s | valid accuracy    0.807 
| end of epoch  10 | time:  9.78s | valid accuracy    0.813 


> LSTM速度慢很多，是RNN的三倍，但收敛速度和精度更好。

## 问题2：LSTM深入

- 首先将LSTM设置为双向LSTM，修改后续对应层的维度，从新训练模型，记录验证集精度；
- 对比使用句子一个token和最后一个token的特征分别用于文本的精度。

In [ ]:
class BILSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size):
        super(BILSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        # self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_w2v))
        
        # 修改下面，bidirectional 为True为双向LSTM
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True)
        self.hidden2label = torch.nn.Linear(hidden_dim*2, label_size)
        
    def forward(self, sentence):
        # print(sentence.shape)
        sentence = torch.transpose(sentence, 1, 0)
        # print(sentence.shape)
        x = self.embeddings(sentence)
        # print(x.shape)
        lstm_out, self.hidden = self.lstm(x)
        # print(lstm_out.shape)
        
        # 修改这里对应token输出的选择，我们先尝试最后一个token对应的输出
        y = self.hidden2label(lstm_out[-1,:,:])
        return y

In [ ]:
num_class = len(lbl)
vocab_size = len(vocab)
emsize = 100
model = BILSTM(vocab_size, emsize, 64, num_class).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5.0, gamma=0.75)

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))

| end of epoch   1 | time: 18.34s | valid accuracy    0.467 
| end of epoch   2 | time: 18.11s | valid accuracy    0.606 
| end of epoch   3 | time: 18.23s | valid accuracy    0.705 
| end of epoch   4 | time: 17.69s | valid accuracy    0.743 
| end of epoch   5 | time: 17.87s | valid accuracy    0.767 
| end of epoch   6 | time: 17.18s | valid accuracy    0.781 
| end of epoch   7 | time: 17.95s | valid accuracy    0.796 
| end of epoch   8 | time: 17.66s | valid accuracy    0.802 
| end of epoch   9 | time: 17.30s | valid accuracy    0.801 
| end of epoch  10 | time: 17.46s | valid accuracy    0.803 


In [ ]:
class BILSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size):
        super(BILSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        # self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_w2v))
        
        # 修改下面，bidirectional 为True为双向LSTM
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True)
        self.hidden2label = torch.nn.Linear(hidden_dim*2, label_size)
        
    def forward(self, sentence):
        # print(sentence.shape)
        sentence = torch.transpose(sentence, 1, 0)
        # print(sentence.shape)
        x = self.embeddings(sentence)
        # print(x.shape)
        lstm_out, self.hidden = self.lstm(x)
        # print(lstm_out.shape)
        
        # 修改这里对应token输出的选择，我们尝试第一个token对应的输出
        y = self.hidden2label(lstm_out[0,:,:])
        return y

In [ ]:
num_class = len(lbl)
vocab_size = len(vocab)
emsize = 100
model = BILSTM(vocab_size, emsize, 64, num_class).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5.0, gamma=0.75)

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))

| end of epoch   1 | time: 17.59s | valid accuracy    0.685 
| end of epoch   2 | time: 17.43s | valid accuracy    0.784 
| end of epoch   3 | time: 17.34s | valid accuracy    0.811 
| end of epoch   4 | time: 17.35s | valid accuracy    0.833 
| end of epoch   5 | time: 17.84s | valid accuracy    0.834 
| end of epoch   6 | time: 17.70s | valid accuracy    0.845 
| end of epoch   7 | time: 18.25s | valid accuracy    0.850 
| end of epoch   8 | time: 18.01s | valid accuracy    0.847 
| end of epoch   9 | time: 17.88s | valid accuracy    0.851 
| end of epoch  10 | time: 17.31s | valid accuracy    0.850 


> token的选择对模型的速度没有影响，但对模型的精度存在影响。如果选择第一个token对应的输出精度更高，因为字符串存在pad操作。“